In [1]:
from __future__ import division

In [2]:
import tensorflow as tf
import tflearn

In [3]:
import numpy as np

In [4]:
from tensorflow.python import debug as tf_debug

In [5]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [6]:
# sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

In [7]:
from tqdm import tqdm

In [8]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
import BayesianNN

In [10]:
bnn = BayesianNN.BayesianFC([4,32,32,1])

In [11]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [12]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [13]:
data_input = list(map(lambda x: [x[0]], data))
data_input_poly = list(map(lambda x: [x[0], x[0]**2, x[0]**3, x[0]**4], data))
data_target = list(map(lambda x: [x[1]], data))

In [14]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    poly = tf.concat(1, [data_input, data_input**2, data_input**3, data_input**4])
    net = tflearn.fully_connected(poly, 32, activation="relu")
    net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [15]:
batch_size = 1000

In [16]:
epoch_loss = []
nn_epoch_loss = []

In [17]:
epochs = 10000

In [18]:
test_set = [[x] for x in np.linspace(-1,1,1000)]
test_set_poly = [[x,x**2,x**3,x**4] for x in np.linspace(-1,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [19]:
op, data_x, data_y, loss, data_output = normal_nn(optim)
sess.run(tf.global_variables_initializer())
for e in tqdm(range(2000)):
    input_batch = data_input
    target_batch = data_target

    _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch, lr:0.001})
    nn_epoch_loss.append(l)

100%|██████████| 2000/2000 [00:03<00:00, 536.58it/s]


In [20]:
nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [21]:
op, data_x, data_y, m_scaling, b_size, loss, kl_loss, data_loss, grads = bnn.update_v2(8, optim)
sess.run(tf.global_variables_initializer())

In [22]:
bnn_lr = 0.001 * 1.0

In [23]:
for e in tqdm(range(epochs)):
#     indices = np.random.randint(low=0, high=1000, size=batch_size)
#     input_batch = [data_input_poly[i] for i in indices]
#     target_batch = [data_target[i] for i in indices]
    input_batch = data_input_poly
    target_batch = data_target
#     ms = (2**(epochs - e - 1))/( (2**epochs) - 1) 
#     ms = 1000/(batch_size)
#     ms = (1000*epochs)/(batch_size*(e+1))
#     ms = (1000)/batch_size
    ms = 0
    
#     grad_nums = [a for (a,b) in grads]
    qw_mean, qw_p, qb_mean, qb_p = sess.run([bnn.qw_means[0], bnn.qw_ps[0], bnn.qb_means[0], bnn.qb_ps[0]])
    _, l, k_loss, d_loss, gg = sess.run([op, loss, kl_loss, data_loss, grads], feed_dict={data_x: input_batch, data_y:target_batch, m_scaling:ms, b_size:batch_size, lr:bnn_lr})
#     print("----------Epoch {}-----------".format(e))
#     print("{}\n{}\n".format(qb_mean, qb_p))
#     print(gg[2:])
#     print()
#     print("KL:{}, Data:{}\n".format(k_loss, d_loss))
    if e % 5 == 0:
        epoch_loss.append(l)   

100%|██████████| 10000/10000 [01:37<00:00, 102.83it/s]


In [24]:
sess.run(bnn.qw_means[0])

array([[-2.55371451,  0.0288885 , -2.76758623, -0.0186172 ,  0.43078631,
        -0.60824162, -0.78586316, -0.96887493, -1.30456913, -1.41930473,
        -1.53285193, -1.0793618 , -1.14194214, -0.90647084, -2.63437247,
        -2.63624525, -0.56276995, -0.97007173, -2.76110363, -0.45388088,
         1.11374187, -1.2953999 , -1.20650518, -1.01856065, -1.35206187,
        -0.92090845, -2.79577708, -2.82595897, -1.2525537 ,  0.49157834,
         1.41384494, -1.21138513],
       [-1.23013771,  1.39174211, -1.10859752,  1.26161218,  1.45251179,
         1.03714359,  1.06249702,  0.59697616,  0.32313183,  0.63495559,
         0.28170416,  0.1846966 ,  0.46552303,  0.75237864, -0.88151908,
        -1.00633252,  0.84551817,  0.31122577, -0.39820856,  1.14299428,
         1.60553122,  0.2993035 ,  0.20569113,  0.5716387 ,  0.18928312,
         0.57211173, -1.24026012, -1.51437187,  0.73196775,  1.52341342,
         1.71071911,  0.2947211 ],
       [ 0.06149322,  1.62586761,  0.28782234,  1.4873

In [25]:
# sess.run(bnn.qw_ps[0])
# grads = tf.gradients(loss, w_vars)
# sess.run(w_vars, feed_dict={data_x: data_input_poly, data_y:target_batch, m_scaling:[0.1]})

In [26]:
# pred_input = tf.placeholder(tf.float32, shape=[None, 4])
# pred_output = bnn.sampled_output(pred_input, 1000)
pred_input, pred_output = bnn.sample(use_mean=True)
predictions = sess.run(pred_output, feed_dict={pred_input: test_set_poly})
var_trials = 500
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(pred_output, feed_dict={pred_input: test_set_poly})], axis=1)

100%|██████████| 499/499 [00:00<00:00, 1081.77it/s]


In [27]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

0.057084281

In [28]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [29]:
predictions.shape

(1000, 500)

In [30]:
p = figure(width=800, height=800, y_range=(-0.5,1.5), x_range=(-1,1))

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

for i in range(0, 200, 25):
    pp = list(map(lambda x: x[i], predictions))
    p.line(test_set_flat, pp, line_width=0.5, color=(255,0,255,0.2))

p.line(data_input_flat, data_target_flat, line_width=1, color="blue")

p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")

# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [31]:
loss_p = figure(width=500, height=500, y_range=(-1,100))

# Loss
loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="purple")

show(loss_p)